In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.3-70b-versatile", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020B9A92CDC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020B9AAD4CA0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Pushkar, I am a Machine Learning Engineer.")])

AIMessage(content="Nice to meet you, Pushkar. It's great to connect with a Machine Learning Engineer. What kind of projects have you been working on lately? Are you more focused on natural language processing, computer vision, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 50, 'total_tokens': 97, 'completion_time': 0.222760093, 'prompt_time': 0.01178092, 'queue_time': 0.01805205, 'total_time': 0.234541013}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-f9f11817-299d-44e7-9079-d348a71c9e57-0', usage_metadata={'input_tokens': 50, 'output_tokens': 47, 'total_tokens': 97})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Pushkar, I am a Machine Learning Engineer."),
        AIMessage(content="Nice to meet you, Pushkar. It's great to connect with a Machine Learning Engineer. What kind of projects have you been working on lately? Are you more focused on natural language processing, computer vision, or something else?"),
        HumanMessage(content="Hey, what's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Pushkar, and you're a Machine Learning Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 118, 'total_tokens': 133, 'completion_time': 0.071062594, 'prompt_time': 0.022492207, 'queue_time': 0.018252017000000002, 'total_time': 0.093554801}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-198f6ab1-d23f-48eb-ac5a-4ad1e1b6da81-0', usage_metadata={'input_tokens': 118, 'output_tokens': 15, 'total_tokens': 133})

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Pushkar, I am a Machine Learning Engineer.")],
    config=config
)
response.content

"Hi Pushkar, nice to meet you. So, you're a Machine Learning Engineer - that's a really cool field. What kind of projects are you working on these days? Are you building models, working with datasets, or exploring new algorithms? I'm curious to know more about your work."

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)
response.content

"Your name is Pushkar, and you're a Machine Learning Engineer."

In [15]:
config1={"configurable":{"session_id":"chat2"}}

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey, my name is John.")],
    config=config1
)
response.content

"Nice to meet you, John. It's great that you're willing to share your name with me. I'll do my best to provide you with helpful and accurate information, and I'll address you by your name from now on. How can I assist you today, John? Do you have any questions or topics you'd like to discuss?"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

'I remember, John. Your name is John.'

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain=prompt|model

In [22]:
chain.invoke({"messages":[HumanMessage(content="Hi, my name is Pushkar.")]})

AIMessage(content="Hello Pushkar, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 59, 'total_tokens': 86, 'completion_time': 0.098181818, 'prompt_time': 0.009795699, 'queue_time': 0.01763153, 'total_time': 0.107977517}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f34901c-42bd-4788-a941-87ed67adb258-0', usage_metadata={'input_tokens': 59, 'output_tokens': 27, 'total_tokens': 86})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Pushkar")],
    config=config
)

response.content

"Hello Pushkar, it's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss."

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain=prompt|model

In [27]:
response=chain.invoke({"messages":[HumanMessage(content="Hi, my name is Pushkar")], "language":"Hindi"})
response.content

'नमस्ते पुष्कर, मैं आपकी मदद करने के लिए यहाँ हूँ। मुझसे क्या पूछना चाहते हैं?'

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am Pushkar")], "language":"Hindi"},
    config=config
)

response.content

'नमस्ते पुष्कर, क्या मैं आपकी किसी तरह से मदद कर सकता हूँ?'

In [32]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")], "language":"Hindi"},
    config=config
)

response.content

'आपका नाम पुष्कर है।'

Managing Conversion History

In [34]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="You are a good assistant."),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no prolem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no prolem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [37]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    |prompt
    |model
)

chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did I ask for?")],
    "language":"English"
    }
)

AIMessage(content='You asked for the solution to the math problem "2+2".', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 115, 'total_tokens': 130, 'completion_time': 0.054545455, 'prompt_time': 0.017908038, 'queue_time': 0.018148591999999998, 'total_time': 0.072453493}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b7c7966-6215-4ae3-b13d-4e914ec0fa49-0', usage_metadata={'input_tokens': 115, 'output_tokens': 15, 'total_tokens': 130})

In [39]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [40]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")], "language":"English"},
    config=config
)

response.content

"I don't know your name. I'm a large language model, I don't have any personal information about you, including your name. I'm here to help answer your questions and provide information, but I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you!"